<a href="https://colab.research.google.com/github/carneiro-fernando/EBAC/blob/main/Exercicios/Modulo_41.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Big Data II - Armazenamento
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Orientação a coluna;</li>
  <li>Particionamento.</li>
</ol>

---

# **Exercícios**

## 1\. Particionamento e orientação a coluna.

Replique as atividades da aula 3 em sua conta da AWS.

1.1. Faça o download do arquivo `crime.csv` (fonte [Kaggle](https://www.kaggle.com/yamqwe/chicago-crimee?select=chicago_crime_2014.csv)).

In [ ]:
!wget https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/crime.csv -q -O crime.csv

1.2. Leia os dados em um DataFrame Pandas e crie a coluna `reference_date`.

In [ ]:
import pandas as pd
from datetime import datetime

#Criando o DataFrame
filename = './crime'
df = pd.read_csv(f'./{filename}.csv')

#Criando a coluna reference_date
df['reference_date'] = df['Date'].apply(lambda date: datetime.strptime(date.split(sep=' ')[0], '%m/%d/%Y').strftime('%Y-%m-%d'))

1.3. Persista o dado no DataFrame em um arquivo csv e em arquivos parquet, estes últimos comprimidos e particionados pela coluna `reference_date`.

In [ ]:
#Salvando no formato CSV
df.to_csv('./crime_enriched.csv', sep=',', index=False)

In [ ]:
#Salvando no formato Parquet
df.to_parquet('./crime', engine='pyarrow', compression='gzip', partition_cols='reference_date')

1.4. Crie os recursos no AWS S3 e AWS IAM e persista os dados em seus respectivos *buckets*.

In [ ]:
from getpass import getpass
print('Digite o usuário')
aws_access_key_id = getpass()
print('Digite a senha')
aws_secret_access_key = getpass()

Digite o usuário
··········
Digite a senha
··········


In [ ]:
#Instalação do SDK Boto3
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 110.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.4 MB/s eta 0:00:00


In [ ]:
#Criação do client (AWS)
import boto3

client = boto3.client(
  's3',
  aws_access_key_id=aws_access_key_id,
  aws_secret_access_key=aws_secret_access_key
)

In [ ]:
#Upload dos arquivos
import os

BUCKET = 'ebac-modulo-41-parquet'

i = 0

for root, dirs, files in os.walk('./crime'):
  elapsed = f'{round(100*i/365, 2)} %'
  print(elapsed)
  for file in files:
    path = os.path.join(root, file)
    bucket_path = '/'.join(path.split(sep='/')[2:])
    client.upload_file(path, BUCKET, bucket_path)
  i = i + 1


0.0 %
0.27 %
0.55 %
0.82 %
1.1 %
1.37 %
1.64 %
1.92 %
2.19 %
2.47 %
2.74 %
3.01 %
3.29 %
3.56 %
3.84 %
4.11 %
4.38 %
4.66 %
4.93 %
5.21 %
5.48 %
5.75 %
6.03 %
6.3 %
6.58 %
6.85 %
7.12 %
7.4 %
7.67 %
7.95 %
8.22 %
8.49 %
8.77 %
9.04 %
9.32 %
9.59 %
9.86 %
10.14 %
10.41 %
10.68 %
10.96 %
11.23 %
11.51 %
11.78 %
12.05 %
12.33 %
12.6 %
12.88 %
13.15 %
13.42 %
13.7 %
13.97 %
14.25 %
14.52 %
14.79 %
15.07 %
15.34 %
15.62 %
15.89 %
16.16 %
16.44 %
16.71 %
16.99 %
17.26 %
17.53 %
17.81 %
18.08 %
18.36 %
18.63 %
18.9 %
19.18 %
19.45 %
19.73 %
20.0 %
20.27 %
20.55 %
20.82 %
21.1 %
21.37 %
21.64 %
21.92 %
22.19 %
22.47 %
22.74 %
23.01 %
23.29 %
23.56 %
23.84 %
24.11 %
24.38 %
24.66 %
24.93 %
25.21 %
25.48 %
25.75 %
26.03 %
26.3 %
26.58 %
26.85 %
27.12 %
27.4 %
27.67 %
27.95 %
28.22 %
28.49 %
28.77 %
29.04 %
29.32 %
29.59 %
29.86 %
30.14 %
30.41 %
30.68 %
30.96 %
31.23 %
31.51 %
31.78 %
32.05 %
32.33 %
32.6 %
32.88 %
33.15 %
33.42 %
33.7 %
33.97 %
34.25 %
34.52 %
34.79 %
35.07 %
35.34 %
35.62 %
35

### **3.3. AWS Athena**

1.5. Crie os recursos no AWS Athena e execute as consultas SQL da aula.

* **Criação das tabelas**
> <img src="https://github.com/carneiro-fernando/EBAC/blob/main/assets/Images/Modulo_41/buckets.png?raw=true">

* **Criação do usuário programático**

> <img src="https://github.com/carneiro-fernando/EBAC/blob/main/assets/Images/Modulo_41/usuario_criado.png?raw=true">

* **Criação da tabela *crime_csv***

> <img src="https://github.com/carneiro-fernando/EBAC/blob/main/assets/Images/Modulo_41/tabela_csv.png?raw=true">

* **Criação da tabela *crime_parquet***

> <img src="https://github.com/carneiro-fernando/EBAC/blob/main/assets/Images/Modulo_41/tabela_parquet.png?raw=true">

#### Queries da base de dados em *CSV*

* **Query 1**

> <img src="https://github.com/carneiro-fernando/EBAC/blob/main/assets/Images/Modulo_41/query_1_csv.png?raw=true">

* **Query 2**

> <img src="https://github.com/carneiro-fernando/EBAC/blob/main/assets/Images/Modulo_41/query_2_csv.png?raw=true">

* **Query 3**

> <img src="https://github.com/carneiro-fernando/EBAC/blob/main/assets/Images/Modulo_41/query_3_csv.png?raw=true">

#### Queries da base de dados em *Parquet*

* **Query 1**

> <img src="https://github.com/carneiro-fernando/EBAC/blob/main/assets/Images/Modulo_41/query_1_parquet.png?raw=true">

* **Query 2**

> <img src="https://github.com/carneiro-fernando/EBAC/blob/main/assets/Images/Modulo_41/query_2_parquet.png?raw=true">

* **Query 3**

> <img src="https://github.com/carneiro-fernando/EBAC/blob/main/assets/Images/Modulo_41/query_3_parquet.png?raw=true">
